#### Importacion de librerias necesarias

Instalar las paqueterias antes de importarlas en el caso de ser necesario

In [1]:
#!pip install sqlalchemy pyodbc #instalar paqueteria si es necesario
#!pip install mysql-connector-python #instalar paqueteria si es necesario
import sqlalchemy as sql
import pandas as pd

#### SQLite

El objetivo de esta seccion es obtener datos desde una BBDD de archivo, en este caso un archivo **SQLite**

In [2]:
#Se crea el motor de BBDD
engine = sql.create_engine('sqlite:///nba_salary.sqlite')

In [3]:
#Se imprimen las tablas que la componen
print(sql.inspect(engine).get_table_names())

['NBA_season1718_salary', 'Seasons_Stats']


Una vez habiendo podido conectarse a la BBDD se crea un **Dataframe** de **Pandas** para poder leer la consulta en este objeto.

In [4]:
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT * FROM NBA_season1718_salary', connection)

df.head()

,X1,Player,Tm,season17_18
0,1.0,Stephen Curry,GSW,34682550.0
1,2.0,LeBron James,CLE,33285709.0
2,3.0,Paul Millsap,DEN,31269231.0
3,4.0,Gordon Hayward,BOS,29727900.0
4,5.0,Blake Griffin,DET,29512900.0


#### MY SQL

En esta seccion se muestra como lograr a una conexion a un motor de base de datos **MySQL**

In [9]:
#SE CREAN QUE CONTIENEN LA INFORMACION QUE SE UTILIZARA PARA CONECTARSE
driver = 'mysql+mysqlconnector'
host = '127.0.0.1'
user = 'root'
password = 'pAs$WoRd'
db_name = 'PokemonDb'

#SE CREA LA CONNECTION STRING(En este caso sin usuario ya que esta autenticado por windows)
connection_string = f"{driver}://{user}:{password}@{host}/{db_name}"

#SE CREA EL OBJETO DE CONEXION
engine = sql.create_engine(connection_string, echo=False)


In [10]:
#MOSTRANDO TABLAS DISPONIBLES EN LA BBDD
sql.inspect(engine).get_table_names()

['pokemon']

Una vez habiendo podido conectarse a la BBDD se crea un **Dataframe** de **Pandas** para poder leer la consulta en este objeto.

In [14]:
#SE ABRE LA CONEXION CON WITH, PARA QUE EL RECURSO SEA LIBERADO AL MOMENTO DE TERMINAR LA CONSULTA
with engine.connect() as connection:
    df = pd.read_sql('SELECT * FROM POKEMON', connection)
    
df.head()

,ID_PKM,NAME,RANK,GENERATION,EVOLVE_FROM,TYPE_1,TYPE_2,HP,ATK,SP_ATK,DEF,SP_DEF,SPEED,TOTAL,HEIGHT,WEIGHT,HABILITIES
0,1,bulbasaur,ordinary,generation-i,nothing,grass,poison,45,49,49,65,65,45,318,7,69,overgrow chlorophyll
1,2,ivysaur,ordinary,generation-i,bulbasaur,grass,poison,60,62,63,80,80,60,405,10,130,overgrow chlorophyll
2,3,venusaur,ordinary,generation-i,ivysaur,grass,poison,80,82,83,100,100,80,525,20,1000,overgrow chlorophyll
3,4,charmander,ordinary,generation-i,nothing,fire,None,39,52,43,60,50,65,309,6,85,blaze solar-power
4,5,charmeleon,ordinary,generation-i,charmander,fire,None,58,64,58,80,65,80,405,11,190,blaze solar-power


In [12]:
#Se genera una query que trae un conteo de los pkm agrupados por generacion y por sus tipos,
    # tambien un promedio de los stat base de los agrupados por ambos tipos y que sean mayores a 500. 

sql_query = '''SELECT 
	            GENERATION,
                TYPE_1,
                TYPE_2,
                COUNT(*) 'Cantidad',
                ROUND(AVG(TOTAL), 2) AS 'Promedio de stat base total'
            FROM POKEMON
            GROUP BY GENERATION, TYPE_1, TYPE_2
            HAVING AVG(TOTAL) > 500
            ORDER BY GENERATION,TYPE_1;'''

In [13]:
with engine.connect() as connection:
    df = pd.read_sql(sql_query, connection)
df.head(10)

,GENERATION,TYPE_1,TYPE_2,Cantidad,Promedio de stat base total
0,generation-i,dragon,flying,1,600.00
1,generation-i,electric,flying,1,580.00
2,generation-i,fire,flying,2,557.00
3,generation-i,ice,flying,1,580.00
4,generation-i,poison,ground,2,505.00
5,generation-i,rock,flying,1,515.00
6,generation-i,water,fighting,1,510.00
7,generation-i,water,flying,1,540.00
8,generation-i,water,ice,3,511.67
9,generation-ii,bug,rock,1,505.00


#### SQL SERVER

En esta seccion se muestra como lograr a una conexion a un motor de base de datos **MS SQLServer**

In [ ]:
#SE CREAN QUE CONTIENEN LA INFORMACION QUE SE UTILIZARA PARA CONECTARSE
driver = 'ODBC Driver 17 for SQL Server'
server = 'SERVER_NAME'
db_name = 'PokemonDb'

#SE CREA LA CONNECTION STRING(En este caso sin usuario ya que esta autenticado por windows)
connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;data_1BASE={db_name};trusted_connection=yes'

#EJEMPLO DE CASO EN QUE EL USUARIO ESTE AUTENTIFICADO(Se debe crear la variable user y password)
#connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;data_1BASE={db_name};UID={USER_NAME};PWD={PASSWORD};&autocommit=true'

#SE CREA URL DE CONECCION COMPATIBLE CON SQLSERVER
connection_url = sql.URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

#SE CREA EL OBJETO DE CONEXION
engine = sql.create_engine(connection_url, use_setinputsizes = False, echo = False)

In [ ]:
#SE MUESTRAN LAS TABLAS DE LA BBDD
print(sql.inspect(engine).get_table_names())

['POKEMON']


Una vez habiendo podido conectarse a la BBDD se crea un **Dataframe** de **Pandas** para poder leer la consulta en este objeto.

In [ ]:
#SE ABRE LA CONEXION CON WITH, PARA QUE EL RECURSO SEA LIBERADO AL MOMENTO DE TERMINAR LA CONSULTA
with engine.connect() as connection:
    df = pd.read_sql('SELECT * FROM POKEMON', con=connection)
    
df.head(3)

,ID_PKM,NAME,RANK,GENERATION,EVOLVE_FROM,TYPE_1,TYPE_2,HP,ATK,SP_ATK,DEF,SP_DEF,SPEED,TOTAL,HEIGHT,WEIGHT,HABILITIES
0,1,bulbasaur,ordinary,generation-i,nothing,grass,poison,45,49,65,49,65,45,318,7,69,overgrow chlorophyll
1,2,ivysaur,ordinary,generation-i,bulbasaur,grass,poison,60,62,80,63,80,60,405,10,130,overgrow chlorophyll
2,3,venusaur,ordinary,generation-i,ivysaur,grass,poison,80,82,100,83,100,80,525,20,1000,overgrow chlorophyll


In [ ]:
sql_query = 'SELECT TYPE_1, TYPE_2, COUNT(*) AS Cantidad, AVG(TOTAL) AS [Promedio de stats base total] FROM POKEMON GROUP BY TYPE_1, TYPE_2 ORDER BY TYPE_1 ASC'

with engine.connect() as connection:
    df = pd.read_sql(sql_query, con=connection)  
df.head()

,TYPE_1,TYPE_2,Cantidad,Promedio de stats base total
0,bug,dark,1,450
1,bug,electric,4,422
2,bug,fairy,2,384
3,bug,fighting,4,552
4,bug,fire,2,455
